# Abalone Rings Regression using Dense Layers

In [25]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [26]:
data = pd.read_csv('data/abalone.csv')

In [27]:
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
data['Sex']

0       2
1       2
2       0
3       2
4       1
       ..
4172    0
4173    2
4174    2
4175    0
4176    2
Name: Sex, Length: 4177, dtype: int64

In [28]:
X = data.drop('Rings', axis=1)
y = data['Rings']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)

In [30]:
class AbaloneRegressionNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AbaloneRegressionNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [31]:
input_size = X_train.shape[1]
hidden_size = 64

net = AbaloneRegressionNet(input_size=input_size, hidden_size=hidden_size)

In [32]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

In [33]:
num_epochs = 1000
batch_size = 32

for epoch in range(num_epochs):
    net.train()
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        outputs = net(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 28231.7734
Epoch [200/1000], Loss: 12.9736
Epoch [300/1000], Loss: 12.9736
Epoch [400/1000], Loss: 12.9736
Epoch [500/1000], Loss: 12.9736
Epoch [600/1000], Loss: 12.9736
Epoch [700/1000], Loss: 12.9736
Epoch [800/1000], Loss: 12.9736
Epoch [900/1000], Loss: 12.9736
Epoch [1000/1000], Loss: 12.9736


In [34]:
net.eval()
with torch.no_grad():
    y_pred = net(X_test_tensor)

y_pred = y_pred.numpy()
y_test = y_test.values.reshape(-1, 1)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae:.4f}')
print(f'RMSE: {mse:.4f}')
print(f'R2 Score: {r2:.4f}')

MAE: 2.3849
RMSE: 10.8301
R2 Score: -0.0005
